In [ ]:
# 1. Importar librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
import os

# Cargar datos procesados
data = pd.read_csv("../data/CMAPSSData/train_FD001_features.csv")

# Definir features y etiqueta
target_col = 'RUL' # Variable objetivo
drop_cols = ['unit', 'time', 'RUL']  # columnas que no queremos como features

X = data.drop(columns=drop_cols) # Matriz de carasteristicas 
y = data[target_col] # Vector de etiquetas

# Separar en train y test
# ⚠ Importante: split por unidades para no mezclar ciclos del mismo motor
units = data['unit'].unique() # Lista de los motores del datasets 
train_units, test_units = train_test_split(units, test_size=0.2, random_state=42) # Dividimos los motores en 2 grupos en train el 80% y en test 20%

# Filtramos para que los motores este en las lista 
X_train = X[data['unit'].isin(train_units)] # Los motores que se usa para entrenar
y_train = y[data['unit'].isin(train_units)]
X_test = X[data['unit'].isin(test_units)] # Los motores que se usa para evaluar 
y_test = y[data['unit'].isin(test_units)]

print(f"Train motors: {len(train_units)}, Test motors: {len(test_units)}")

# Entrenar modelo
model = RandomForestRegressor( # Modelo de ensamble basado en arbol de decision
    n_estimators=200, # Numero de arboles del modelo, mas arboles mas estable pero lenta
    max_depth=15, # Numero de profundidad de cada arbol, numero de divisiones de las ramas 
    random_state=42, # Semilla aleatoria de reproducibilidad
    n_jobs=-1 # Numero de nucleo de cpu que usa
)
model.fit(X_train, y_train)

# Evaluar modelo
y_pred = model.predict(X_test) # Usamos el modelo que entrenamos para predecir la rul y_pred -> Array con las predicciones

# Usamos numpy sqrt por falla en la opcion squared
rmse = np.sqrt(mean_squared_error(y_test, y_pred)) # Root Mean Squared error -> Igual que mean pero penaliza mas los errores grandes 
mae = mean_absolute_error(y_test, y_pred) # Mean Absolute error -> Promedio de la diferencia entre la prediccion y el valor real

print(f"RMSE: {rmse:.2f}") # Mostrar resultado con 2 difras
print(f"MAE:  {mae:.2f}")

# Guardar modelo
os.makedirs("../models", exist_ok=True)  # Crea el directorio en la raiz
joblib.dump(model, "../models/rf_model_fd001.pkl", compress = 3 ) # Guarda los resultado en un archivo

print("Modelo guardado en ../models/rf_model_fd001.pkl")

Train motors: 80, Test motors: 20
RMSE: 34.38
MAE:  24.18
Modelo guardado en ../models/rf_model_fd001.pkl
